In [1]:
import pandas as pd
import glob
import sys
import json
import csv

In [33]:
allSeqsWithOneAnn_df = pd.read_csv("/data/sara/semantic-segmentation-pytorch/data/AllSeqsWithOneAnn2beCleaned",sep=":", names=['path', 'label'])
labels_df = pd.read_csv("/data/sara/semantic-segmentation-pytorch/data/all_image_lavel_labeled_20210211_cleaned.uniq", sep=":", names=['path', 'label'])

In [2]:
filename = '/data/sara/semantic-segmentation-pytorch/data/AllSeqsWithOneAnn2beCleaned'

In [35]:
def label(filename): #file name is a file that each line is path: cluster-name
    with open(filename + '_labeled', 'w') as f:
        lines = open(filename, 'r').readlines()
        for line in lines:
            imgname = line.split(':')[0].split('/')[-1]
            seqname = line.split(':')[1].strip()
            format_ = imgname.split('.')[-1]
            try:
                found = labels_df[labels_df['path'] == imgname.replace('png', 'JPG')]
                new_line = '/home/mousavi/da1/icputrd/arf/mean.js/public/sara_img/' + \
                            found['path'].values[0][:3] + "/" + found['path'].values[0].replace('JPG', format_) + \
                            ":" + seqname + "-"+ found['label'].values[0] + "\n"

                f.write(new_line)
            except:
                pass
    
label(filename)

In [3]:
def odgt_format(filename):
    df_seqs = pd.read_csv(filename + "_labeled.uniq", sep=":", names=['img', 'label'])
    with open(filename + "_labeled.uniq.odgt", "w") as fw:
        for path in glob.glob("new_train_noBack_annotations/" + "*.png"):
            img_name = path.split("/")[-1][:-4]
            match = df_seqs[df_seqs['img'].str.contains(img_name)]
            if match.shape[0] > 0:
                match = match.reset_index(drop = True)
                labels = match.label.unique()
                #print(len(labels))
                for label in labels:
                    seq_match = df_seqs[df_seqs['label']==label]
                    seq_match = seq_match.reset_index(drop = True)
                    if seq_match.shape[0] >= 2:
                        for row in seq_match.iterrows():
                            img_names = ["/usb/seq_data_for_mit_code/" + img_name + '.JPG']
                            seg_names = ["new_train_noBack_annotations/" + img_name + '.png']
                            secon_img_name = row[1].img.split('/')[-1]
                            if 'png' not in secon_img_name:
                                img_names.append("/usb/seq_data_for_mit_code/" + row[1].img.split('/')[-1].replace('.icon',''))
                                seg_names.append("new_train_noBack_annotations/" + img_name + '.png')
                            if len(list(set(img_names))) == 2 and len(list(set(seg_names))):
                                new_line = {}
                                new_line["fpath_img"], new_line["fpath_segm"], new_line["width"], new_line["height"] = \
                                        img_names, seg_names, 600, 400
                                json.dump(new_line, fw)
                                fw.write('\n')
                            
            

In [4]:
odgt_format(filename)

In [2]:
df_seqs = pd.read_csv("data/sequence_data/AllMnaullyImgLevelLabeledSeqsWith10SBPPlusAllSeqsWithOneAnn2beCleaned", 
                      sep=":", 
                      names=['img', 'label'])
seq_size = 10

## get sequences with len 10 in flat format

In [39]:
sys.stdout = open("seqsWLen10WOneAnnFlat", "w")
res_df = pd.DataFrame()
seq_counts = 0
for path in glob.glob("new_train_noBack_annotations/" + "*.png"):
    img_name = path.split("/")[-1][:-4]
    match = df_seqs[df_seqs['img'].str.contains(img_name)]
    if match.shape[0] > 0:
        match = match.reset_index(drop = True)
        labels = match.label.unique()
        for label in labels:
            seq_match = df_seqs[df_seqs['label'].str.contains(label)]
            seq_match = seq_match.reset_index(drop = True)
            if seq_match.shape[0] >= seq_size - 1:
                print("/usb/seq_data_for_mit_code/{}.JPG: new_train_noBack_annotations/{}.png".
                     format(img_name, img_name))
                res = seq_match.sample(seq_size - 1).reset_index(drop = True)
                for row in res.iterrows():
                    print("/usb/seq_data_for_mit_code/{}: new_train_noBack_annotations/{}.png".
                          format(row[1].img.split('/')[-1].replace('.icon',''), img_name))
                seq_counts += 1



## get sequences with len 10 in odgt format

In [5]:
with open("seqsWLen10WOneAnn.odgt", "w") as fw:
    seq_counts = 0
    for path in glob.glob("new_train_noBack_annotations/" + "*.png"):
        img_name = path.split("/")[-1][:-4]
        match = df_seqs[df_seqs['img'].str.contains(img_name)]
        if match.shape[0] > 0:
            match = match.reset_index(drop = True)
            labels = match.label.unique()
            for label in labels:
                seq_match = df_seqs[df_seqs['label'].str.contains(label)]
                seq_match = seq_match.reset_index(drop = True)
                if seq_match.shape[0] >= seq_size - 1:
                    res = seq_match.sample(seq_size - 1).reset_index(drop = True)
                    img_names = ["/usb/seq_data_for_mit_code/" + img_name + '.JPG']
                    seg_names = ["new_train_noBack_annotations/" + img_name + '.png']
                    for row in res.iterrows():
                        img_names.append("/usb/seq_data_for_mit_code/" + row[1].img.split('/')[-1].replace('.icon',''))
                        seg_names.append("new_train_noBack_annotations/" + img_name + '.png')
                    new_line = {}
                    new_line["fpath_img"], new_line["fpath_segm"], new_line["width"], new_line["height"] = \
                                img_names, seg_names, 600, 400
                    json.dump(new_line, fw)
                    fw.write('\n')
                    seq_counts += 1


## get mit format data for sequences with size 2 to 10

In [17]:
def format_seq(inputfile, length):
    with open(inputfile, "r") as f_r, open("new_cct_seq_" + str(length), 'w') as f_cct, open("new_mit_seq_" + str(length), 'w') as f_mit:
        sequence_data = f_r.readlines()
        write = csv.writer(f_cct, delimiter = ' ')
        for line in sequence_data:
            new_line = json.loads(line)
            new_line['fpath_img'] = new_line['fpath_img'][:length]
            new_line['fpath_segm'] = new_line['fpath_segm'][:length]
            json.dump(new_line, f_mit)
            f_mit.write('\n')
            
            write.writerow(new_line['fpath_img'][:length])
            

In [4]:
for i in range(2, 11):
    format_seq("data/seqsWLen10WOneAnn.odgt", i)

In [37]:
col_names = ["seq_{}".format(i) for i in range(1, 11)]
df_new_labeled_seq = pd.read_csv('newly_labaled_imgs_to_fix_imbalance_in_seq_format', sep=" ", names=col_names)
df_new_labeled_img = pd.read_csv('newly_labaled_imgs_to_fix_imbalance', names=[col_names[0]])
newly_labaled_imgs = df_new_labeled_img.seq_1.values
with open("newly_labaled_imgs_to_fix_imbalance_in_seq_format.odgt", "w") as fw:
    for img in newly_labaled_imgs:
        try:
            match = df_new_labeled_seq[df_new_labeled_seq.seq_1 == img]
            img_names = list(match.values[0])
            img_names = ["/usb/seq_data_for_mit_code/" + i for i in img_names]
            seg_names = ["new_train_noBack_annotations/" + img.replace('JPG', 'png')] * 10
            new_line = {}
            new_line["fpath_img"], new_line["fpath_segm"], new_line["width"], new_line["height"] = \
                img_names, seg_names, 600, 400
            json.dump(new_line, fw)
            fw.write('\n')
        except:
            print(img)

27800129.10.JPG
a4800127.23.JPG


In [38]:
for i in range(2, 11):
    format_seq("newly_labaled_imgs_to_fix_imbalance_in_seq_format.odgt", i)

In [33]:
seg_names

['new_train_noBack_annotations/00b00329.13.png',
 'new_train_noBack_annotations/00b00329.13.png',
 'new_train_noBack_annotations/00b00329.13.png',
 'new_train_noBack_annotations/00b00329.13.png',
 'new_train_noBack_annotations/00b00329.13.png',
 'new_train_noBack_annotations/00b00329.13.png',
 'new_train_noBack_annotations/00b00329.13.png',
 'new_train_noBack_annotations/00b00329.13.png',
 'new_train_noBack_annotations/00b00329.13.png',
 'new_train_noBack_annotations/00b00329.13.png']